In [ ]:
!unzip /content/drive/MyDrive/Colab-Notebooks/data-20231115T183312Z-001.zip

Archive:  /content/drive/MyDrive/Colab-Notebooks/data-20231115T183312Z-001.zip
  inflating: data/valid_Y.npy        
  inflating: data/train_Y.npy        
  inflating: data/valid_bboxes.npy   
  inflating: data/train_bboxes.npy   
  inflating: data/valid_seg.npy      
  inflating: data/valid_X.npy        
  inflating: data/train_seg.npy      
  inflating: data/train_X.npy        


In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.210 🚀 Python-3.10.12 torch-2.1.0+cu118 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.7/107.7 GB disk)


In [ ]:
#object detection

In [ ]:
import torch
import numpy as np
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor, Resize
from PIL import Image

class CustomYOLODataset(Dataset):
    def __init__(self, images_file, labels_file, bboxes_file, target_size=(640, 640)):
        self.images = np.load(images_file, allow_pickle=True)
        self.labels = np.load(labels_file, allow_pickle=True)
        self.bboxes = np.load(bboxes_file, allow_pickle=True)
        self.transform = ToTensor()
        self.resize = Resize(target_size)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.fromarray(self.images[idx])

        # Convert grayscale to RGB if needed
        if image.mode != 'RGB':
            image = image.convert('RGB')

        # Resize the image to the target size
        image = self.resize(image)

        label = torch.tensor(self.labels[idx])
        bbox = torch.tensor(self.bboxes[idx])

        # Apply YOLO-specific preprocessing here, if needed

        image = self.transform(image)

        return image, label, bbox


In [ ]:
import torch
from torch.utils.data import DataLoader
#from custom_yolo_dataset import CustomYOLODataset
#from models.yolo import Model  # Import your YOLO model architecture
from ultralytics import YOLO

# Paths to your training data files
train_images_file = '/content/data/train_X.npy'
train_labels_file = '/content/data/train_Y.npy'
train_bboxes_file = '/content/data/train_bboxes.npy'

# Paths to your validation data files
valid_images_file = '/content/data/valid_X.npy'
valid_labels_file = '/content/data/valid_Y.npy'
valid_bboxes_file = '/content/data/valid_bboxes.npy'

# Create instances of the custom datasets
train_dataset = CustomYOLODataset(train_images_file, train_labels_file, train_bboxes_file)
valid_dataset = CustomYOLODataset(valid_images_file, valid_labels_file, valid_bboxes_file)

# Define DataLoaders
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# Instantiate the YOLO model
model = YOLO('yolov8n.pt')  # You need to replace this with your YOLO model architecture

# Define loss function and optimizer

100%|██████████| 6.23M/6.23M [00:00<00:00, 38.3MB/s]


In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50  # Adjust as needed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# for epoch in range(num_epochs):
#     # Training
#     model.train()
#     for batch in train_dataloader:
#         images, labels, bboxes = batch
#         images = images.to(device)
#         labels = labels.to(device)
#         bboxes = bboxes.to(device)

#         # Forward pass
#         outputs = model(images)
#         #print(outputs)
#         # Assuming outputs is a list of tensors, you might need to concatenate or process them
#         # based on the specifics of your model's output format

#         # Calculate loss (modify this based on your actual loss calculation)
#         loss = 0
#         for scale_output in outputs:
#             loss += criterion(scale_output, labels)

#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     # Validation
#     model.eval()
#     with torch.no_grad():
#         for batch in valid_dataloader:
#             images, labels, bboxes = batch
#             images = images.to(device)
#             labels = labels.to(device)
#             bboxes = bboxes.to(device)

#             # Forward pass (no optimization during validation)
#             outputs = model(images)

#             # Assuming outputs is a list of tensors, you might need to concatenate or process them
#             # based on the specifics of your model's output format

#             # Calculate validation loss (modify this based on your actual loss calculation)
#             val_loss = 0
#             for scale_output in outputs:
#                 val_loss += criterion(scale_output, labels)

#     print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}')

engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coco8.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=None, workspace=4, nms=False, lr0=0.01

TypeError: ignored

In [ ]:
# import torch
# from torch.utils.data import DataLoader
# #from custom_yolo_dataset import CustomYOLODataset
# #from models.yolo import Model  # Import your YOLO model architecture
# from ultralytics import YOLO

# # Paths to your training data files
# train_images_file = '/content/data/train_X.npy'
# train_labels_file = '/content/data/train_Y.npy'
# train_bboxes_file = '/content/data/train_bboxes.npy'

# # Paths to your validation data files
# valid_images_file = '/content/data/valid_X.npy'
# valid_labels_file = '/content/data/valid_Y.npy'
# valid_bboxes_file = '/content/data/valid_bboxes.npy'

# # Create instances of the custom datasets
# train_dataset = CustomYOLODataset(train_images_file, train_labels_file, train_bboxes_file)
# valid_dataset = CustomYOLODataset(valid_images_file, valid_labels_file, valid_bboxes_file)

# # Define DataLoaders
# batch_size = 16
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# # Instantiate the YOLO model
# model = YOLO('yolov8n.pt')  # You need to replace this with your YOLO model architecture

# # Define loss function and optimizer
# criterion = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# num_epochs = 50  # Adjust as needed
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)
# num_classes = 80

# def process_yolo_outputs(outputs):
#     # Concatenate or process your model outputs here
#     # This depends on the specifics of your YOLO model's architecture
#     processed_outputs = torch.cat(outputs, dim=1)
#     return processed_outputs


# for epoch in range(num_epochs):
#     # Training
#     model.train()
#     for batch in train_dataloader:
#         images, labels, bboxes = batch
#         images = images.to(device)
#         labels = labels.to(device)
#         bboxes = bboxes.to(device)

#         # Forward pass
#         outputs = model(images)

#         # Process the model outputs to get the final predictions
#         # This step depends on your specific YOLO model architecture
#         # processed_outputs = process_yolo_outputs(outputs)

#         # # Calculate loss
#         # loss = criterion(processed_outputs, labels)

#         # # Backward pass and optimization
#         # optimizer.zero_grad()
#         # loss.backward()
#         # optimizer.step()


# # Assuming `outputs` is the list of YOLO model outputs
# # Assuming `labels` is the ground truth labels for the batch

# # Process YOLO model outputs (assuming it's a list of tensors)
#         processed_outputs = process_yolo_outputs(outputs)

#         # Assuming `processed_outputs` is a tensor with shape (batch_size, num_anchors, num_classes + 4)
#         # where the last dimension represents class probabilities and bounding box coordinates

#         # Extract class probabilities and bounding box coordinates
#         class_probs = processed_outputs[:, :, :num_classes]
#         box_coords = processed_outputs[:, :, num_classes:]

#         # Extract ground truth class labels and bounding box coordinates
#         gt_class_labels = labels[:, :, 0]
#         gt_box_coords = labels[:, :, 1:]

#         # Convert ground truth box coordinates to YOLO format if needed

#         # Calculate classification loss using cross-entropy loss
#         classification_loss = torch.nn.CrossEntropyLoss()(class_probs, gt_class_labels)

#         # Calculate localization loss using a suitable regression loss (e.g., smooth L1 loss)
#         localization_loss = torch.nn.SmoothL1Loss()(box_coords, gt_box_coords)

#         # Total loss
#         total_loss = classification_loss + localization_loss

#         # Backward pass and optimization
#         optimizer.zero_grad()
#         total_loss.backward()
#         optimizer.step()


#     # Validation
#     model.eval()
#     with torch.no_grad():
#         for batch in valid_dataloader:
#             images, labels, bboxes = batch
#             images = images.to(device)
#             labels = labels.to(device)
#             bboxes = bboxes.to(device)

#             # Forward pass (no optimization during validation)
#             outputs = model(images)

#             # Process the model outputs for validation
#             processed_outputs = process_yolo_outputs(outputs)

#             # Calculate validation loss
#             val_loss = criterion(processed_outputs, labels)

#     print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}')

engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coco8.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=None, workspace=4, nms=False, lr0=0.01,

TypeError: ignored

In [ ]:
from torch.nn import CrossEntropyLoss, SmoothL1Loss

classification_loss_fn = CrossEntropyLoss()
localization_loss_fn = SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50  # Adjust as needed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        images, labels, bboxes = batch
        images = images.to(device)
        labels = labels.to(device)
        bboxes = bboxes.to(device)

        # Forward pass
        outputs = model(images)
        boxes = outputs[0].boxes
        for box in boxes:

        # Extract predicted bounding box coordinates and class probabilities
          pred_bboxes = box.xyxy[:, :4]
          pred_class_probs = box.xyxy[:, 4]

          # Extract ground truth bounding box coordinates and class labels
          gt_bboxes = labels[:, :, 1:]
          gt_class_labels = labels[:, :, 0].long()

          # Calculate classification loss using cross-entropy loss
          classification_loss = classification_loss_fn(pred_class_probs, gt_class_labels)

          # Calculate localization loss using smooth L1 loss
          localization_loss = localization_loss_fn(pred_bboxes, gt_bboxes)

          # Total loss
          total_loss = classification_loss + localization_loss

        # Backward pass and optimization
          optimizer.zero_grad()
          total_loss.backward()
          optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        total_val_loss = 0
        for val_batch in valid_dataloader:
            val_images, val_labels, val_bboxes = val_batch
            val_images = val_images.to(device)
            val_labels = val_labels.to(device)
            val_bboxes = val_bboxes.to(device)

            # Forward pass for validation
            val_outputs = model(val_images)
            boxes = outputs[0].boxes
            for box in boxes:

            # Extract predicted bounding box coordinates and class probabilities
              val_pred_bboxes = box.xyxy[:, :4]
              val_pred_class_probs = box.xyxy[:, 4]

              # Extract ground truth bounding box coordinates and class labels
              val_gt_bboxes = val_labels[:, :, 1:]
              val_gt_class_labels = val_labels[:, :, 0].long()

              # Calculate classification loss for validation
              val_classification_loss = classification_loss_fn(val_pred_class_probs, val_gt_class_labels)

              # Calculate localization loss for validation
              val_localization_loss = localization_loss_fn(val_pred_bboxes, val_gt_bboxes)

              # Total validation loss
              total_val_loss += val_classification_loss + val_localization_loss

    avg_val_loss = total_val_loss / len(valid_dataloader)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {total_loss.item():.4f}, Validation Loss: {avg_val_loss.item():.4f}')

# Save the trained model if needed
torch.save(model.state_dict(), 'yolo_model.pth')

Streaming output truncated to the last 5000 lines.
0: 640x640 (no detections), 1: 640x640 (no detections), 2: 640x640 (no detections), 3: 640x640 (no detections), 4: 640x640 (no detections), 5: 640x640 (no detections), 6: 640x640 (no detections), 7: 640x640 (no detections), 8: 640x640 (no detections), 9: 640x640 (no detections), 10: 640x640 (no detections), 11: 640x640 (no detections), 12: 640x640 (no detections), 13: 640x640 (no detections), 14: 640x640 (no detections), 15: 640x640 (no detections), 4976.1ms
Speed: 0.0ms preprocess, 311.0ms inference, 17.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 1: 640x640 (no detections), 2: 640x640 (no detections), 3: 640x640 (no detections), 4: 640x640 (no detections), 5: 640x640 (no detections), 6: 640x640 (no detections), 7: 640x640 (no detections), 8: 640x640 (no detections), 9: 640x640 (no detections), 10: 640x640 (no detections), 11: 640x640 (no detections), 12: 640x640 (no detections), 13: 640x640 (no det